In [23]:
import pandas as pd

In [24]:
train_df = pd.read_csv('./data/raw/train.csv')
test_df = pd.read_csv('./data/raw/test.csv')
combine = [train_df, test_df]
numerical_features = ['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
categorical_features = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# Cleaning the data

Let's turn the categorical features into numerical values.

In [25]:
# Start with the sex
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map({
        'female':1,
        'male':0
    }).astype(int)

# Now the embarked
for dataset in combine:
    dataset.dropna(subset=['Embarked'], inplace=True)    
    dataset['Embarked'] = dataset['Embarked'].map({
        'S':2,
        'Q':1,
        'C':0
    }).astype(int)


See if there is a null values

In [26]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

Since there are many values in the feature cabin, it is the best to remove it.

In [27]:
for dataset in combine:
    dataset.drop('Cabin', axis=1, inplace=True)

Managing missing values in the Age column.
1. See if there are others features correlated to the age.
2. Use the correlated features to fill our data.

In [28]:
train_df.corr(numeric_only=True)['Age'].sort_values(ascending=False)

Age            1.000000
Fare           0.093143
PassengerId    0.033681
Embarked      -0.032565
Survived      -0.082446
Sex           -0.099037
Parch         -0.187896
SibSp         -0.307351
Pclass        -0.365902
Name: Age, dtype: float64

An accurate way of guessing missing values is to use other correlated features. In our case we note correlation among Age, Gender, and Pclass. Guess Age values using median values for Age across sets of Pclass and Gender feature combinations. So, median Age for Pclass=1 and Gender=0, Pclass=1 and Gender=1, and so on...



In [29]:
import numpy as np

def fill_missing_ages(dataset, guess_ages):
    for i in range(2):  # 0: Male, 1: Female
        for j in range(3):  # 1: 1st class, 2: 2nd class, 3: 3rd class
            # Filter rows based on Sex and Pclass, and drop missing Age values
            guess_df = dataset[(dataset['Sex'] == i) & 
                               (dataset['Pclass'] == j + 1)]['Age'].dropna()
            
            # Calculate the median age and round to the nearest 0.5
            age_guess = guess_df.median()
            guess_ages[i, j] = round(age_guess * 2) / 2
            
    for i in range(2):
        for j in range(3):
            # Fill missing Age values with the corresponding guess_ages value
            dataset.loc[(dataset['Age'].isnull()) & 
                        (dataset['Sex'] == i) & 
                        (dataset['Pclass'] == j + 1), 'Age'] = guess_ages[i, j]

    dataset['Age'] = dataset['Age'].astype(int)

# Initialize the array to store age guesses
guess_ages = np.zeros((2, 3))

# Apply the function to each dataset in the combine list
for dataset in combine:
    fill_missing_ages(dataset, guess_ages)

# Display the first few rows of the training dataset to verify
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,7.2500,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,2
4,5,0,3,"Allen, Mr. William Henry",0,35,0,0,373450,8.0500,2


Since Ticket and PassengerId are just Id's and aren't correlated with our goal (survived), I'm gonna remove them.

In [30]:
for dataset in combine:
    dataset.drop(['Ticket','Name'], axis=1, inplace=True)

In [31]:
train_df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22,1,0,7.2500,2
1,2,1,1,1,38,1,0,71.2833,0
2,3,1,3,1,26,0,0,7.9250,2
3,4,1,1,1,35,1,0,53.1000,2
4,5,0,3,0,35,0,0,8.0500,2
...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27,0,0,13.0000,2
887,888,1,1,1,19,0,0,30.0000,2
888,889,0,3,1,21,1,2,23.4500,2
889,890,1,1,0,26,0,0,30.0000,0


First, fix the missign values from our data.

In [32]:
test_df.loc[(test_df['Fare'].isnull()),'Fare'] = test_df['Fare'].median()

In [33]:
train_df.to_csv('./data/clean_df.csv')
test_df.to_csv('./data/test_clean.csv')